In [49]:
import numpy as np
import xarray as xr
import pandas as pd 
import sys 
from EchoPro import EchoPro
epro_2019 = EchoPro(init_file_path='./config_files/initialization_config.yml',
                    survey_year_file_path='./config_files/survey_year_2019_config.yml',
                    source=3,
                    bio_data_type=1,
                    age_data_status=1, 
                    exclude_age1=True)

A check of the initialization file needs to be done!
A check of the survey year file needs to be done!
Loading biological data ...


/Users/brandonreyes/UW_work/EchoPro_work/EchoPro/EchoPro/load_biological_data/load_biological_data.py:214: UserWarning: The Sex column contains values {3}, converting values greater than 2 to 2 and values less than 2 to 1
  warnings.warn(
/Users/brandonreyes/UW_work/EchoPro_work/EchoPro/EchoPro/load_biological_data/load_biological_data.py:214: UserWarning: The Sex column contains values {3}, converting values greater than 2 to 2 and values less than 2 to 1
  warnings.warn(
/Users/brandonreyes/UW_work/EchoPro_work/EchoPro/EchoPro/load_biological_data/load_biological_data.py:63: UserWarning: The Sex column contains values {3}, converting values greater than 2 to 2 and values less than 2 to 1
  warnings.warn(
/Users/brandonreyes/UW_work/EchoPro_work/EchoPro/EchoPro/load_biological_data/load_biological_data.py:63: UserWarning: The Sex column contains values {3}, converting values greater than 2 to 2 and values less than 2 to 1
  warnings.warn(
/Users/brandonreyes/UW_work/EchoPro_work/EchoP

In [2]:
print(sys.getsizeof(epro_2019.length_df))
# epro_2019.length_df.head()

277624


In [3]:
print(epro_2019.length_ds_bran.nbytes)
# epro_2019.length_ds_bran

75048


In [4]:
print(epro_2019.length_ds_wu_jung.nbytes)
# epro_2019.length_ds_wu_jung

63448


In [5]:
# print(np.all(epro_2019.length_df['n'].to_numpy() == epro_2019.length_ds_bran.n.values))
# print(np.allclose(epro_2019.length_df['meanlen'].to_numpy(), epro_2019.length_ds_bran.meanlen.values))
# print(np.allclose(epro_2019.length_df['stdlen'].to_numpy(), epro_2019.length_ds_bran.stdlen.values))
# print(np.allclose(epro_2019.length_df['TS_lin'].to_numpy(), epro_2019.length_ds_bran.TS_lin.values))
# print(np.allclose(epro_2019.length_df['TS_log'].to_numpy(), epro_2019.length_ds_bran.TS_log.values))
# print(np.allclose(epro_2019.length_df['nM'].to_numpy(), epro_2019.length_ds_bran.nM.values))
# print(np.allclose(epro_2019.length_df['nF'].to_numpy(), epro_2019.length_ds_bran.nF.values))

In [6]:
# print(np.all(epro_2019.length_df['n'].to_numpy() == epro_2019.length_ds_wu_jung.n.values))
# print(np.allclose(epro_2019.length_df['meanlen'].to_numpy(), epro_2019.length_ds_wu_jung.meanlen.values))
# print(np.allclose(epro_2019.length_df['stdlen'].to_numpy(), epro_2019.length_ds_wu_jung.stdlen.values))
# print(np.allclose(epro_2019.length_df['TS_lin'].to_numpy(), epro_2019.length_ds_wu_jung.TS_lin.values))
# print(np.allclose(epro_2019.length_df['TS_log'].to_numpy(), epro_2019.length_ds_wu_jung.TS_log.values))
# print(np.allclose(epro_2019.length_df['nM'].to_numpy(), epro_2019.length_ds_wu_jung.nM.values))
# print(np.allclose(epro_2019.length_df['nF'].to_numpy(), epro_2019.length_ds_wu_jung.nF.values))

In [7]:
# df = pd.read_excel(epro_2019.params['data_root_dir'] + epro_2019.params['filename_catch_US'])
# ds = epro_2019.load_bio.process_catch_data_ds(df)
# ds

In [8]:
# df = epro_2019.load_bio.process_catch_data_df(df)
# df.head()

In [9]:
# ds.species_code.sel(haul=73)[0: ds.no_null_length.sel(haul=73).values]

In [10]:
# df['Species_Code'].loc[73]

In [12]:
print(epro_2019.params['ave_len_wgt_M'])
print(epro_2019.params['ave_len_wgt_F'])
print(epro_2019.params['ave_len_wgt_ALL'])

0.43899002636583884
0.5179108950801983
0.4781804012273613


In [29]:
# Create catch table with 
# Columns   1-3:   'Trawl number'   'Abundance'     'Biomass' 
epro_2019.final_table_catch = epro_2019.catch_df[epro_2019.catch_df['Species_Code'] == epro_2019.params['species_code_ID']]

In [47]:

# TODO: make strata_df a variable of epro_2019
# load stratification file 
stratification_index = 1
if stratification_index != 1 and stratification_index != 0:
    raise NotImplementedError(f"stratification_index of {stratification_index} has not been implemented!")
else:
    
    if stratification_index == 1: 
        strata_df = pd.read_excel(epro_2019.params['data_root_dir'] + epro_2019.params['filename_strata'], sheet_name='Base KS')
        strata_df = strata_df[['Year', 'Cluster name', 'Haul', 'wt']].copy()
        
        # set data types of dataframe
        strata_df = strata_df.astype({'Year': int, 'Cluster name': int, 'Haul': int, 'wt': np.float64})
        
        strata_df.set_index('Haul', inplace=True)
        strata_df.sort_index(inplace=True)
        
    else:
        strata_df = pd.read_excel(epro_2019.params['data_root_dir'] + epro_2019.params['filename_strata'], sheet_name='INPFC')
        strata_df = strata_df[['Year', 'INPFC', 'Haul', 'wt']].copy()
        
        # set data types of dataframe
        strata_df = strata_df.astype({'Year': int, 'INPFC': int, 'Haul': int, 'wt': np.float64})
        
        strata_df.set_index('Haul', inplace=True)
        strata_df.sort_index(inplace=True)
    
strata_df.head()


,Year,Cluster name,wt
Haul,,,
1,2019,1,1.0
3,2019,2,1.0
7,2019,1,1.0
8,2019,1,1.0
9,2019,1,1.0


In [157]:

# TODO: make final_table_trawl a variable of epro_2019
if stratification_index == 1:
    selected_columns = ['Haul', 'Transect', 'EQ_Latitude', 'EQ_Longitude', 'Cluster name', 'Average_Bottom_Depth', 
                        'Surface_Temperature', 'Gear_Temperature', 'Length', 'Sex', 'Age', 'Weight', 'Frequency',
                        'Average_Footrope_Depth', 'Weight_In_Haul']
    final_table_trawl = pd.DataFrame(columns=selected_columns)
elif stratification_index == 0:
    selected_columns = ['Haul', 'Transect', 'EQ_Latitude', 'EQ_Longitude', 'INPFC', 'Average_Bottom_Depth', 
                        'Surface_Temperature', 'Gear_Temperature', 'Length', 'Sex', 'Age', 'Weight', 'Frequency',
                        'Average_Footrope_Depth', 'Weight_In_Haul']
    final_table_trawl = pd.DataFrame(columns=selected_columns)
else: 
    raise NotImplementedError(f"stratification_index of {stratification_index} has not been implemented!")
    
catch_specific = epro_2019.catch_df[epro_2019.catch_df['Species_Code'] == epro_2019.params['species_code_ID']]

# get all unique Haul values accross all of the data 
full_haul = np.unique(np.concatenate([strata_df.index.values, epro_2019.gear_df.index.values, 
                                      catch_specific.index.values, epro_2019.length_ds_wu_jung.Haul.values, 
                                      epro_2019.trawl_df.index.values, epro_2019.specimen_df.index.unique().values]))

full_haul

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
       201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213,
       214, 215, 216, 217, 218, 219, 220, 221, 222])

In [162]:
from functools import reduce

temp = epro_2019.length_ds_wu_jung.Frequency.to_dataframe().reset_index().dropna(subset='Frequency').set_index('Haul')
    
# TODO: change strata_df['Cluster name'] to be more inclusive 
data_frames_length = [temp, catch_specific, epro_2019.gear_df, epro_2019.trawl_df, strata_df['Cluster name']]
df_merged_length = reduce(lambda  left,right: pd.merge(left,right,on=['Haul'], how='outer'), data_frames_length)

df_merged_length['Age'] = np.nan
df_merged_length['Weight'] = np.nan

# TODO: change strata_df['Cluster name'] to be more inclusive 
data_frames_specimen = [epro_2019.specimen_df, catch_specific, epro_2019.gear_df, epro_2019.trawl_df, strata_df['Cluster name']]
df_merged_specimen = reduce(lambda  left,right: pd.merge(left,right,on=['Haul'], how='outer'), data_frames_specimen)

df_merged_specimen['Frequency'] = np.int64(1) # TODO: check to make sure if adding a frequency column makes sense with Chu


df_merged_length[selected_columns[1:]]
# df_merged_specimen[selected_columns[1:]]

# TODO: combine df_merged_length and df_merged_specimen 


,Transect,EQ_Latitude,EQ_Longitude,Cluster name,Average_Bottom_Depth,Surface_Temperature,Gear_Temperature,Length,Sex,Age,Weight,Frequency,Average_Footrope_Depth,Weight_In_Haul
Haul,,,,,,,,,,,,,,
1,1.0,34.403645,-120.656847,1.0,390.565671,13.024097,7.940186,20.0,1.0,NaN,NaN,2.0,335.762771,56.25
1,1.0,34.403645,-120.656847,1.0,390.565671,13.024097,7.940186,20.0,2.0,NaN,NaN,5.0,335.762771,56.25
1,1.0,34.403645,-120.656847,1.0,390.565671,13.024097,7.940186,21.0,1.0,NaN,NaN,22.0,335.762771,56.25
1,1.0,34.403645,-120.656847,1.0,390.565671,13.024097,7.940186,21.0,2.0,NaN,NaN,13.0,335.762771,56.25
1,1.0,34.403645,-120.656847,1.0,390.565671,13.024097,7.940186,22.0,1.0,NaN,NaN,53.0,335.762771,56.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,113.0,53.064167,-130.787150,NaN,123.000000,NaN,8.140000,NaN,NaN,NaN,NaN,NaN,82.000000,NaN
217,121.0,54.525100,-131.150783,NaN,150.000000,NaN,7.750000,NaN,NaN,NaN,NaN,NaN,91.000000,NaN
219,127.0,54.628817,-132.871967,NaN,260.000000,NaN,6.150000,NaN,NaN,NaN,NaN,NaN,204.000000,NaN


In [142]:
temp.head()

,Length,Sex,Frequency
Haul,,,
1,20.0,1,2.0
1,20.0,2,5.0
1,21.0,1,22.0
1,21.0,2,13.0
1,22.0,1,53.0


In [143]:
epro_2019.catch_df[epro_2019.catch_df['Species_Code'] == epro_2019.params['species_code_ID']].head()

,Species_Code,Species_Name,Number_In_Haul,Weight_In_Haul
Haul,,,,
1,22500,Pacific hake,769.0,56.25
3,22500,Pacific hake,866.0,46.25
7,22500,Pacific hake,848.0,51.90
8,22500,Pacific hake,1415.0,103.10
9,22500,Pacific hake,173.0,11.85


In [138]:
epro_2019.gear_df.head()

,Average_Footrope_Depth,Surface_Temperature,Gear_Temperature,Average_Wireout,Transect,Net_Height
Haul,,,,,,
1,335.762771,13.024097,7.940186,880.300000,1.0,19.000000
2,NaN,NaN,NaN,NaN,NaN,NaN
3,227.011792,14.423039,9.160076,504.336538,4.0,20.945961
4,172.438149,14.431567,9.760245,411.980000,4.0,24.000000
5,140.932086,14.548182,9.430227,306.101911,5.0,20.768069


In [140]:
epro_2019.trawl_df.head()

,Haul_Type,Performance_Code,Duration,Distance_Fished,Stratum,EQ_Latitude,EQ_Longitude,Average_Bottom_Depth,Vessel_Log_Start,Vessel_Log_Stop
Haul,,,,,,,,,,
1,8,0.0,0 days 00:15:31,0.887076,0,34.403645,-120.656847,390.565671,NaN,NaN
2,8,9.0,NaT,NaN,0,NaN,NaN,550.000000,NaN,NaN
3,8,0.0,0 days 00:15:16,0.877471,0,34.898575,-120.907500,248.893706,NaN,NaN
4,8,0.0,0 days 00:05:03,0.278713,0,34.894292,-120.876322,200.023129,NaN,NaN
5,8,0.0,0 days 00:15:27,0.896190,0,35.062545,-121.454705,432.352672,NaN,NaN


In [144]:
strata_df.head()

,Year,Cluster name,wt
Haul,,,
1,2019,1,1.0
3,2019,2,1.0
7,2019,1,1.0
8,2019,1,1.0
9,2019,1,1.0


In [141]:
epro_2019.specimen_df.head()

,Sex,Length,Weight,Specimen_Number,Age
Haul,,,,,
1,1,24.0,0.08,100620381.0,1.0
1,1,23.0,0.06,100620412.0,1.0
1,1,22.0,0.06,100620413.0,1.0
1,1,21.0,0.06,100620414.0,1.0
1,2,22.0,0.06,100620415.0,1.0


In [52]:
selected_columns = ['Transect', 'EQ_Latitude', 'EQ_Longitude', 'Cluster name', 'Average_Bottom_Depth', 'Surface_Temperature', 
                    'Gear_Temperature', 'Average_Footrope_Depth', 'Weight_In_Haul'] #, 'length', 'sex', 'age']

final_table_trawl = pd.concat([catch_temp, epro_2019.gear_df, epro_2019.trawl_df, strata_df], axis=1)[selected_columns]

final_table_trawl.head()


,Transect,EQ_Latitude,EQ_Longitude,Cluster name,Average_Bottom_Depth,Surface_Temperature,Gear_Temperature,Average_Footrope_Depth,Weight_In_Haul
Haul,,,,,,,,,
1,1.0,34.403645,-120.656847,1.0,390.565671,13.024097,7.940186,335.762771,56.25
3,4.0,34.898575,-120.907500,2.0,248.893706,14.423039,9.160076,227.011792,46.25
7,7.0,35.393610,-121.158050,1.0,329.915149,15.721684,7.827097,197.570074,51.90
8,8.0,35.558283,-121.433993,1.0,608.990611,15.252079,7.836350,341.447245,103.10
9,8.0,35.561963,-121.299280,1.0,283.547586,14.354921,8.637334,271.710300,11.85


In [66]:
epro_2019.length_ds_wu_jung.Frequency.sel(Haul=1).dropna(dim='Length').to_dataframe().reset_index().set_index('Haul')

,Length,Sex,Frequency
Haul,,,
1,20.0,1,2.0
1,20.0,2,5.0
1,21.0,1,22.0
1,21.0,2,13.0
1,22.0,1,53.0
1,22.0,2,38.0
1,23.0,1,47.0
1,23.0,2,38.0
1,24.0,1,37.0


In [67]:
epro_2019.specimen_df.head()

,Sex,Length,Weight,Specimen_Number,Age
Haul,,,,,
1,1,24.0,0.08,100620381.0,1.0
1,1,23.0,0.06,100620412.0,1.0
1,1,22.0,0.06,100620413.0,1.0
1,1,21.0,0.06,100620414.0,1.0
1,2,22.0,0.06,100620415.0,1.0
